# Claude Opus 4.6 on Amazon Bedrock — 機能チュートリアル

このノートブックでは、**Claude Opus 4.6** (`global.anthropic.claude-opus-4-6-v1`) で利用可能な主要機能を Amazon Bedrock 経由で実際に動かしながら学びます。

## 取り扱う機能

- Adaptive Thinking + Effort
    - リクエストの複雑さに応じて Claude が自律的に思考の深さを調整する機能。effortパラメータで思考量のガイダンスを指定可能                                                                                                
- 1M コンテキストウィンドウ (Beta)
    - デフォルト 200K トークンのコンテキストウィンドウを 1Mトークンに拡張し、大量のドキュメントやコードベースを一度に処理可能にする                                     
- 最大 128K の出力トークン
    - 出力トークンの上限を最大 128K トークンまで指定可能。長文のコード生成や翻訳、レポート作成に有用                  
- 構造化出力 - JSON Outputs
    - LLM のレスポンスを指定した JSON Schema に厳密に準拠させ、型安全でパースエラーのない出力を保証する機能
- Compaction (Beta)
    - 長い会話でコンテキストウィンドウのトークン数がしきい値を超えた際に、会話を自動要約して圧縮し、会話を継続可能にする機能

---
## セットアップ

### 前提条件

- Python 3.9+
- boto3 1.42+
- AWS 認証情報が設定済み (`AWS_PROFILE` or デフォルト認証チェーン)

#### Amazon Bedrockの利用に必要なIAMの権限について

本サンプルでは、以下のIAMのActionを実行する権限が必要です。初回モデルアクセス時は後述するAWS Marketplaceの権限も必要です。

- bedrock:InvokeModel
- bedrock:InvokeModelWithResponseStream

####  Anthropic モデルを初めて使う場合

Anthropicのモデルを初めて利用する場合、AWSアカウント、もしくはOrganizationごとに一回、ユースケース情報の提出が必要です。

- Amazon Bedrockコンソールのモデルカタログページから、「Submit use case details」ボタンを押し、フォームを入力
- 送信後、即座にアクセスが有効化されます

####  AWS Marketplace 権限について

Bedrockのサードパーティモデル（Anthropic含む）は、モデルごとに、初回呼び出し時に自動でアカウントに有効化されます。この自動有効化には以下のIAM権限が必要です：

- aws-marketplace:Subscribe
- aws-marketplace:ViewSubscriptions

> **Note:** 一度有効化されれば、以降の呼び出しにはMarketplace権限は不要です

#### EULA
モデル初回呼び出し時に該当モデルの[エンドユーザーライセンス契約（EULA）]に同意したものとみなされます。

詳細はこちらをご覧ください: [Manage access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/model-access.html)

まずは共通で使うライブラリのインポートと Bedrock クライアントの初期化を行います。

In [ ]:
# 最新のboto3を利用
%pip install --upgrade boto3

In [ ]:
import boto3
import copy
import json
import os

# 利用するAWSのProfile名を指定してください。
AWS_PROFILE = ""
AWS_REGION = os.environ.get("AWS_REGION", "ap-northeast-1")
MODEL_ID = os.environ.get("MODEL_ID", "global.anthropic.claude-opus-4-6-v1")

if AWS_PROFILE:
    session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION)
    # print(f"Profile:  {AWS_PROFILE}")
else:
    session = boto3.Session(region_name=AWS_REGION)

# 現在の AWS Identity を確認
sts = session.client("sts", region_name=AWS_REGION)
identity = sts.get_caller_identity()
# print(f"Account:  {identity['Account']}")
# print()

print(f"リージョン: {AWS_REGION}")
print(f"モデル ID:  {MODEL_ID}")

def truncate_for_display(result, max_chars=900):
    """invoke_model レスポンスの表示用ユーティリティ。
    長い text/thinking ブロックを切り詰め、signature を省略してJSON全体の構造を確認しやすくする。"""
    display = copy.deepcopy(result)
    for block in display.get("content", []):
        if block.get("type") == "text":
            t = block.get("text", "")
            if len(t) > max_chars:
                block["text"] = t[:max_chars] + f"... (truncated, total: {len(t)} chars)"
        elif block.get("type") == "thinking":
            t = block.get("thinking", "")
            if len(t) > max_chars:
                block["thinking"] = t[:max_chars] + f"... (truncated, total: {len(t)} chars)"
            if "signature" in block:
                block["signature"] = "(omitted)"
    return display

### read_timeout について

boto3 (botocore) の HTTP クライアントには **`read_timeout`** というパラメータがあり、デフォルトは **60 秒** です。これはサーバーからのレスポンスデータ読み取りを待つ最大時間を意味します。

Bedrock で大規模モデルを使用する場合、**60 秒では足りず `ReadTimeoutError` が発生** することがあります。

これを回避するには `botocore.config.Config` の `read_timeout` を明示的に延長します。

> **Note:** ストリーミング API (`invoke_model_with_response_stream`) ではチャンクが逐次到着するため read_timeout の問題は起きにくいですが、非ストリーミングの `invoke_model` では出力全体の生成完了まで待つ必要があるため注意が必要です。

**参考:**
- [Amazon Bedrock の大規模モデル呼び出しにおける読み取りタイムアウトの解決方法](https://repost.aws/ja/knowledge-center/bedrock-large-model-read-timeouts)
- [botocore Config リファレンス](https://botocore.amazonaws.com/v1/documentation/api/latest/reference/config.html)

In [ ]:
from botocore.config import Config

bedrock = session.client(
    "bedrock-runtime",
    region_name=AWS_REGION,
    config=Config(read_timeout=300),  # 300秒 (5分) に延長
)

---
## モデルへのアクセス方法 — InvokeModel API と Converse API

Bedrock で Claude を呼び出すには主に 2 つの API があります。

| API | 特徴 |
|-----|------|
| **Converse API** (`bedrock.converse`) | モデル非依存の統一インターフェース。Bedrock 上の全モデルで共通の形式を利用可能。基本的な対話・ストリーミング・ツール利用に推奨。一部のClaudeの機能の利用に制限あり |
| **InvokeModel API** (`bedrock.invoke_model`) | 基盤モデルの推論を呼び出すためのAPI。各モデルプロバイダー固有の仕様に合わせてパラメーターを指定することが可能であり、Anthropic Messages API 形式のリクエストボディを直接送信できる |

このサンプルでは、機能をシンプルに紹介するため、InvokeModel API を利用する例で説明する。Converse API での実装については末尾のAppendixに記載。


In [ ]:
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 256,
    "messages": [
        {
            "role": "user",
            "content": "日本の首都はどこですか？"
        }
    ],
})

response = bedrock.invoke_model(
    modelId=MODEL_ID,
    body=body
)

result = json.loads(response["body"].read())
text = result["content"][0]["text"]
print(f"質問: 日本の首都はどこですか？")
print(f"回答: {text}")

### レスポンス構造の確認

InvokeModel API のレスポンスにはトークン使用量などのメタデータも含まれています。

In [ ]:
# メタデータの確認
print(json.dumps(truncate_for_display(result), ensure_ascii=False, indent=2))
print(f"停止理由:     {result['stop_reason']}")
print(f"入力トークン: {result['usage']['input_tokens']}")
print(f"出力トークン: {result['usage']['output_tokens']}")

---
## 1. Adaptive Thinking + Effort

**Adaptive Thinking** は Claude Opus 4.6 で推奨される Extended Thinking の利用方法です。

従来の Extended Thinking では固定の thinking token budget を手動設定する必要がありましたが、Adaptive Thinking ではリクエストの複雑さに応じて Claude が自律的に **「考えるかどうか」「どの程度深く考えるか」** を判断します。

### Effort パラメータ

`output_config.effort` を組み合わせることで、思考量のガイダンスを与えられます。

| effort level | 挙動 |
|-------------|------|
| `max` | 常に思考し、思考の深さに制約なし (**Opus 4.6 専用**) |
| `high` (デフォルト) | 常に思考する。複雑なタスクで深い推論を行う |
| `medium` | 適度に思考する。非常に簡単な質問では思考をスキップする場合がある |
| `low` | 思考を最小限にする。速度優先の簡単なタスクで思考をスキップ |

### 課金について

Adaptive Thinking で使われる thinking トークンは **出力トークンとして課金** され、`max_tokens` の対象に含まれます。

### 実装ポイント

- `"thinking": {"type": "adaptive"}` と `"output_config": {"effort": "xxx"}` を指定

**参考:**
- [Adaptive Thinking (Anthropic)](https://platform.claude.com/docs/en/build-with-claude/adaptive-thinking)

In [ ]:
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 4096,
    "messages": [
        {
            "role": "user",
            "content": "LLMにおいて、なぜChain of Thoughtが有効なのか、LLMの技術トレンドの中での位置付けを踏まえて考えて下さい",
        }
    ],
    "thinking": {"type": "adaptive"},
    "output_config": {"effort": "medium"},
})

response = bedrock.invoke_model(modelId=MODEL_ID, body=body)
result = json.loads(response["body"].read())

# レスポンス全体の構造を確認 (長いブロックは切り詰め表示)
# thinking 有効時は content 配列に text / thinking ブロックが混在する
print(json.dumps(truncate_for_display(result), ensure_ascii=False, indent=2))

### 出力 (thinking あり)

Adaptive Thinking が有効で、かつ Claude が思考を行った場合、レスポンスは以下のような構造になります:

```json
{
  "content": [
    {
      "type": "text",
      "text": "\n\n"                    // 空の text (thinking の前に挿入される場合がある)
    },
    {
      "type": "thinking",
      "thinking": "...",               // 内部思考
      "signature": "..."              // 改ざん防止用署名 (マルチターン時にそのまま返す)
    },
    {
      "type": "text",
      "text": "回答..."             // ユーザーに返す回答
    }
  ],
  "stop_reason": "end_turn",
  "usage": {
    "input_tokens": ...,
    "output_tokens": ...               // thinking トークンも output_tokens に含まれる
  }
}
```


### Effort レベルの比較

同じ質問を `low` effort で投げてみて、思考量の違いを確認します。  
簡単な質問を `low` で投げると thinking ブロックが生成されない場合があります。

In [ ]:
body_low = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 4096,
    "messages": [
        {
            "role": "user",
            "content": "日本の首都はどこですか？",
        }
    ],
    "thinking": {"type": "adaptive"},
    "output_config": {"effort": "low"},
})

response_low = bedrock.invoke_model(modelId=MODEL_ID, body=body_low)
result_low = json.loads(response_low["body"].read())

print(json.dumps(truncate_for_display(result_low), ensure_ascii=False, indent=2))


### 出力 (thinking なし)

`effort=low` かつ簡単な質問の場合、Claude は思考をスキップします。レスポンスは以下のようにシンプルになります:

```json
{
  "content": [
    {
      "type": "text",   // thinking ブロックなし、text のみ
      "text": "回答..."
    }   
  ],
  "stop_reason": "end_turn",
  "usage": {"input_tokens": ..., "output_tokens": ...}
}
```

thinking ブロックが生成されない分 `output_tokens` が大幅に少なく、レイテンシ・コストともに低くなります。

---
## 2. 1M コンテキストウィンドウ

Claude Opus 4.6 のデフォルトは **200K (20万) トークン** コンテキストウィンドウですが、beta flag を指定することで **1M (100万) トークン** のコンテキストウィンドウを利用できます。

1M トークンというのは、ハリー・ポッターと賢者の石 の本 3冊弱 にも相当する量であり、大量のドキュメント分析、長大なコードベースの理解、膨大なログの検索などに有用です。

### 実装ポイント

- `"anthropic_beta": ["context-1m-2025-08-07"] `を指定
- モデル ID は 200K トークン と 1M トークン モデルで同一。

### 注意事項

- 1M トークンの処理には今回のような処理でも 数10秒掛かることがあります。
- 入力トークン課金が大きくなるため、prompt caching との併用を検討すること。
- 200K トークン モデルに対して、それ以上のトークン数のpromptを入力した場合、 `ValidationException: prompt is too long: 200001 tokens > 200000 maximum` のようなエラーとなる。

**参考:**
- [Context Windows - 1M token context window (Anthropic)](https://platform.claude.com/docs/en/build-with-claude/context-windows#1-m-token-context-window)
- [Context Window と Extended Thinking (Anthropic)](https://platform.claude.com/docs/en/build-with-claude/context-windows#the-context-window-with-extended-thinking)
- [Bedrock Count Tokens](https://docs.aws.amazon.com/bedrock/latest/userguide/count-tokens.html)

### Needle-in-Haystack テスト

1M トークン モデルの挙動確認のため、大量のダミーテキストの中にパスフレーズを埋め込み、モデルに見つけさせるテストを行います。

#### Count Tokens による事前見積もり

大量のテキストを送信する前に `bedrock.count_tokens()` で入力トークン数を事前に確認できます。API 呼び出しなしでトークン数が分かるため、コスト見積もりやコンテキストウィンドウ超過の防止に有用です。

- `count_tokens` は **ベースモデル ID** (`anthropic.claude-xxxx`) のみ対応。Cross-region inference profile (`global.anthropic.claude-xxx`) 等は指定不可
- 結果は実際の `usage.input_tokens` より約 20 トークン多い (内部システムトークンを含む。課金対象外)
- `count_tokens` 自体は無料 (レート制限あり)


In [ ]:
# 検索対象のパスフレーズ (needle)
NEEDLE = "OPUS46-BEDROCK"

def build_long_text(lines_needed: int = 5000) -> str:
    """指定行数のダミーテキストを生成し、中間に needle を埋め込む"""
    line = "Amazon Web Services は、コンピューティング、ストレージ、データベース、分析、ネットワーキング、モバイル、デベロッパーツール、管理ツール、IoT、セキュリティ、エンタープライズアプリケーションなど、グローバルなクラウドベースの製品を幅広く提供しています。これらはオンデマンドで数秒で利用でき、従量制料金が適用されます。データウェアハウスからデプロイツール、ディレクトリ、コンテンツ配信まで、200 を超える AWS サービスが利用可能です。\n"
    mid = lines_needed // 2

    lines = []
    for i in range(lines_needed):
        if i == mid:
            lines.append(f"パスフレーズは {NEEDLE} です。\n")
        lines.append(line)

    return "".join(lines)

text = build_long_text()
prompt = f"{text}\n\n上記テキストに含まれる「秘密のパスフレーズ」を正確に抜き出してください。"

# 1M context window モデルを利用する際は anthropic_beta: context-1m-2025-08-07 flag が必須
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "anthropic_beta": ["context-1m-2025-08-07"],
    "max_tokens": 256,
    "messages": [{"role": "user", "content": prompt}],
})

# count_tokens で事前にトークン数を確認
# ベースモデルID のみ対応 (global.から始まるグローバルクロスリージョン推論プロファイルの利用は不可)

COUNT_TOKENS_MODEL_ID = MODEL_ID.replace("global.", "")
response = bedrock.count_tokens(
    modelId=COUNT_TOKENS_MODEL_ID,
    input={"invokeModel": {"body": body}},
)
print(f"推定入力トークン: {response['inputTokens']:,} tokens")

In [ ]:
response = bedrock.invoke_model(modelId=MODEL_ID, body=body)
result = json.loads(response["body"].read())

answer = result["content"][0]["text"]
print(f"回答: {answer}")

usage = result["usage"]
print(f"入力トークン: {usage['input_tokens']:,}")
print(f"出力トークン: {usage['output_tokens']:,}")


---
## 3. 最大 128K の出力トークン

Claude Opus 4.6 では `max_tokens` を最大 **128,000 トークン** まで指定できます。  
長文のコード生成、翻訳、レポート作成などに有用です。

### 実装ポイント

- Opus 4.6 では出力トークンの上限が 64K トークンから 128K トークンに拡大されました。
- `invoke_model` / `invoke_model_with_response_stream` の body に `"max_tokens": 128000` を指定
- Converse API でも `inferenceConfig.maxTokens` で指定可能
- Streamingを利用しない場合は、Bedrock clientを作成する際の`read_timeout`の値を十分に大きな値 (数10分以上) に設定してください。

### Streamingを利用した 128K の出力トークン の機能検証

InvokeModelWithResponseStream APIを利用して、ストリームで約100Kトークンのテキストを出力させます。

100K トークンの出力には、Claude Opus 4.6で数10分掛かるのでご留意下さい。


In [ ]:
# 出力ファイルパス
OUTPUT_FILE = "output/output_128k_stream.txt"

body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 128000,
    "thinking": {"type": "adaptive"},
    "output_config": {"effort": "max"},
    "messages": [
        {
            "role": "user",
            "content": """あなたは長編SF作家です。10万文字の長編小説を必ず一回の出力で執筆してください。

## フェーズ1: 構想
最初に以下を検討してください：
- テーマ・世界観・時代設定
- 主要キャラクター（5〜8名）
- プロットの骨子

## フェーズ2: 章立て
章立てを設計：
- 大章10個（タイトル・役割・目標）
- 各大章にサブ章8〜10個（シーン概要）

## フェーズ3: 執筆ルール
1. 各サブ章は約1000文字
2. 大章開始時に全体章立てを再掲：
   【全体章立て】第1章←現在 / 第2章 / ... / 第10章
3. 形式：【第X章 - X-Y】タイトル → 本文
4. 前後の連続性・伏線を意識
""",
        }
    ],
})

# ストリーミングで呼び出し
response = bedrock.invoke_model_with_response_stream(modelId=MODEL_ID, body=body)

current_block_type = None

# バッファリングを無効化 (buffering=1: 行バッファ) してリアルタイムにファイルへ反映
with open(OUTPUT_FILE, "w", encoding="utf-8", buffering=1) as f:
    for event in response["body"]:
        chunk = json.loads(event["chunk"]["bytes"])
        event_type = chunk["type"]

        if event_type == "content_block_start":
            current_block_type = chunk.get("content_block", {}).get("type")
            if current_block_type == "thinking":
                f.write("=== Thinking Block ===\n")
                print("=== Thinking Block ===")

        elif event_type == "content_block_stop":
            if current_block_type == "thinking":
                f.write("\n=== End Thinking Block ===\n\n")
                print("\n=== End Thinking Block ===\n")
            current_block_type = None

        elif event_type == "content_block_delta":
            delta = chunk["delta"]
            if delta["type"] == "text_delta":
                text = delta["text"]
                print(text, end="", flush=True)
                f.write(text)
            elif delta["type"] == "thinking_delta":
                thinking = delta["thinking"]
                print(thinking, end="", flush=True)
                f.write(thinking)

        elif event_type == "message_delta":
            stop_reason = chunk["delta"].get("stop_reason", "")
            output_tokens = chunk["usage"]["output_tokens"]
            print(f"\n\n--- ストリーム完了 ---")
            print(f"停止理由:     {stop_reason}")
            print(f"出力トークン: {output_tokens:,}")
            print(f"出力ファイル: {OUTPUT_FILE}")

        elif event_type == "message_start":
            input_tokens = chunk["message"]["usage"]["input_tokens"]
            print(f"[入力トークン: {input_tokens:,}]\n")

---
## 5. 構造化出力 (JSON Outputs)

### Structured Outputs とは

**Structured Outputs** は、LLMのレスポンスを **指定したスキーマに厳密に準拠した形式に制約する** 機能です。

通常の LLM 出力はあくまでも自由形式のテキストなので、プログラムで後処理する際に以下のような問題が起こり得ます:

- JSON のパースエラー（不正な構文、閉じ括弧の欠落など）
- 必須フィールドの欠落
- 型の不一致（数値が期待される場所に文字列が返る等）
- スキーマ違反に対するリトライ処理の必要性

Structured Outputs は **Constrained Decoding（制約付きデコーディング）** により、上記を解決する機能です。

### 2 つのアプローチ

Structured Outputs には **2 つの補完的な機能** があります:

| 機能 | パラメータ | 制約対象 | 用途 |
|------|-----------|---------|------|
| **JSON outputs** | `output_config.format` | Claude の **レスポンス形式** (Claude が何を返すか) | データ抽出、レポート生成、API レスポンス整形 |
| **Strict tool use** | `strict: true` (Tool 定義内) | Tool の **入力パラメータ** (Claude がどう関数を呼ぶか) | エージェントワークフロー、型安全な関数呼び出し |

これらは独立して使うことも、**同じリクエスト内で併用** することもできます。

ここのサンプルでは、 **JSON outputs** を利用します。

JSON Outputs はデータ抽出や、分類・タグ付け、後段のAPIとの連携等「**Claude の出力をそのまま後続プログラムに渡したい**」場面で有効です:

#### 実装ポイント

- JSON outputsを利用する場合は、`output_config.format` にスキーマを指定します。
- **初回リクエストの遅延**: スキーマごとに Grammar コンパイルが走るため初回はやや遅い。コンパイル結果は **24 時間キャッシュ** され、以降は高速
- Adaptive Thinking と併用可能。ただしレスポンスに `text("\n\n")` → `thinking` → `text(JSON)` の 3 ブロックが含まれるため、**最後の text ブロック** から JSON をパースする必要がある

**参考:**
- [Structured Outputs (Anthropic)](https://platform.claude.com/docs/en/build-with-claude/structured-outputs)
- [Bedrock Structured Output](https://docs.aws.amazon.com/bedrock/latest/userguide/structured-output.html)

In [ ]:
# 出力スキーマ定義
BOOK_SCHEMA = {
    "type": "object",
    "properties": {
        "title": {"type": "string"},
        "author": {"type": "string"},
        "year": {"type": "integer"},
        "genre": {
            "type": "string",
            "enum": ["fiction", "non-fiction", "science", "history", "philosophy"],
        },
        "summary": {"type": "string"},
        "recommended": {"type": "boolean"},
    },
    "required": ["title", "author", "year", "genre", "summary", "recommended"],
    "additionalProperties": False,
}

print("スキーマ定義:")
print(json.dumps(BOOK_SCHEMA, ensure_ascii=False, indent=2))

In [ ]:
# output_config.format を使用して構造化出力を取得
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1024,
    "messages": [
        {
            "role": "user",
            "content": "「2001年宇宙の旅」という本について教えてください。",
        }
    ],
    "output_config": {
        "format": {
            "type": "json_schema",
            "schema": BOOK_SCHEMA,
        }
    },
})

response = bedrock.invoke_model(modelId=MODEL_ID, body=body)
result = json.loads(response["body"].read())

# 最後の text ブロックから JSON をパース
raw_text = result["content"][-1]["text"]
parsed = json.loads(raw_text)

print("構造化出力 (JSON):")
print(json.dumps(parsed, ensure_ascii=False, indent=2))

---
## 6. Compaction (会話圧縮)

**Compaction** は、長い会話セッションでコンテキストウィンドウを効率的に管理するための機能です。

会話が設定したトークンしきい値に近づくと、Claude が自動的に会話の要約を生成し、圧縮されたコンテキストで会話を継続できます。長時間のエージェンティックタスクや対話型アプリケーションに有用です。

### 動作フロー

1. 入力トークン数がトリガーしきい値を超えたことを検知
2. 現在の会話の要約を生成
3. 要約を含む compaction ブロックを作成
4. 圧縮されたコンテキストを使って応答を継続、もしくは、一時停止してクライアントに処理を委譲

### 実装ポイント

- beta flag: **`compact-2026-01-12`** (必須)
- `context_management.edits` 配列内に compaction 設定を記述
- `pause_after_compaction=true` の場合、compaction 発生後に `stop_reason="compaction"` が返る。クライアント側で compaction ブロックを取り出し、直近のメッセージと合わせて再リクエストする

#### 詳細なパラメータ

| パラメータ | 説明 |
|-----------|------|
| `type` | `"compact_20260112"` を指定 |
| `trigger` | compaction を発動するタイミング。デフォルトは 150,000 トークン。最低 50,000 以上 |
| `pause_after_compaction` | `true` にすると compaction 後に `stop_reason="compaction"` で一時停止。クライアント側でハンドリング可能 |
| `instructions` | カスタム要約プロンプト (以下のデフォルトプロンプトを完全に置き換えます) |


デフォルトプロンプト:

```You have written a partial transcript for the initial task above. Please write a summary of the transcript. The purpose of this summary is to provide continuity so you can continue to make progress towards solving the task in a future context, where the raw history above may not be accessible and will be replaced with this summary. Write down anything that would be helpful, including the state, next steps, learnings etc. You must wrap your summary in a <summary></summary> block.```

**参考:**
- [Compaction (Anthropic)](https://platform.claude.com/docs/en/build-with-claude/compaction)
- [Bedrock Claude Compaction](https://docs.aws.amazon.com/bedrock/latest/userguide/claude-messages-compaction.html)

In [ ]:
from botocore.config import Config

# Compaction デモ用に read_timeout を延長したクライアントを作成
bedrock_compaction = session.client(
    "bedrock-runtime",
    region_name=AWS_REGION,
    config=Config(read_timeout=1000),
)

# 会話履歴
compaction_messages = []

def _invoke_compaction(request_messages, pause_after_compaction=True):
    """Compaction 対応の invoke_model 呼び出し"""
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "anthropic_beta": ["compact-2026-01-12"],
        "max_tokens": 15000,
        "messages": request_messages,
        "context_management": {
            "edits": [
                {
                    "type": "compact_20260112",
                    "trigger": {"type": "input_tokens", "value": 50000},
                    "pause_after_compaction": pause_after_compaction,
                }
            ]
        },
        "thinking": {"type": "adaptive"},
        "output_config": {"effort": "high"},
    })

    response = bedrock_compaction.invoke_model(modelId=MODEL_ID, body=body)
    result = json.loads(response["body"].read())

    usage = result.get("usage", {})
    print(f"[tokens] input: {usage.get('input_tokens', '?'):,}, output: {usage.get('output_tokens', '?'):,}")
    return result

def chat_with_compaction(user_message):
    """ユーザーメッセージを送信し、compaction 発生時はハンドリングして応答を返す"""
    compaction_messages.append({"role": "user", "content": user_message})

    result = _invoke_compaction(compaction_messages, pause_after_compaction=True)

    # compaction が発生して一時停止した場合のハンドリング
    if result.get("stop_reason") == "compaction":
        compaction_block = result["content"][0]

        print("  [COMPACTION] Compaction が発生しました！")
        print("  レスポンス構造:")
        print(json.dumps(truncate_for_display(result), ensure_ascii=False, indent=2))

        # 直近の 2 メッセージを保持して再構築
        preserved = compaction_messages[-2:] if len(compaction_messages) >= 2 else compaction_messages[:]
        messages_after = [
            {"role": "assistant", "content": [compaction_block]}
        ] + preserved

        # 圧縮済みコンテキストで再度呼び出し
        result = _invoke_compaction(messages_after, pause_after_compaction=False)

        compaction_messages.clear()
        compaction_messages.extend(messages_after)

    # 最終応答を履歴に追加
    compaction_messages.append({"role": "assistant", "content": result["content"]})

    text_parts = [b["text"] for b in result["content"] if b["type"] == "text"]
    return "\n".join(text_parts).strip()

#### マルチターン会話で Compaction を体験

Web アプリケーションの実装について 9 往復の対話を行います。会話が長くなると入力トークンがしきい値 (50,000) を超え、自動的に Compaction が発動します。

`[COMPACTION]` のログが表示されたら、会話が圧縮されて継続されていることを確認してください。対話が終わるまで20分程掛かる場合があります。

In [ ]:
prompts = [
    "PythonでユーザーのCRUD操作ができるREST APIを作る際の実装について教えて",
    "データベースに接続してデータを永続化できるように修正して",
    "認証機能を追加して。ログインとトークン検証ができるようにして",
    "このAPIを使うフロントエンドを作って。ログイン画面とユーザー一覧画面を実装して",
    "フロントエンドからAPIを呼び出せるように書き足して",
    "バックエンドをDockerコンテナ化して",
    "docker-composeでAPI、フロントエンド、DBをまとめて起動できるようにして",
    "CDKでAWSにこのアプリをホストするインフラを構築するコードを書いて",
    "GitHub Actionsでmainへのpush時に自動デプロイするワークフローを書いて",
]

for i, prompt in enumerate(prompts, 1):
    print(f"\n{'='*60}")
    print(f"[{i}/10] User: {prompt}")
    print(f"{'='*60}")
    response_text = chat_with_compaction(prompt)
    # 応答の先頭 500 文字をプレビュー表示
    preview = response_text[:500] + "..." if len(response_text) > 500 else response_text
    print(f"\nAssistant: {preview}\n")


### 実行時の挙動

この例では、後半の質問で入力トークンが `trigger` の 50,000 を超え、**Compaction が自動発動** することが期待されます。
このサンプルでは以下のようなイメージの挙動を取ります。

```
[1/9]  input:  1,000
[2/9]  input:  8,000
[3/9]  input: 16,000
[4/9]  input: 25,000
[5/9]  input: 34,000
[6/9]  input: 43,000
[7/9]  input: 52,000  →  50,000 超過し Compaction が発動
        ├─ stop_reason="compaction" で一時停止
        ├─ クライアントが compaction ブロック + 直近メッセージをセットして再リクエスト
        └─ input: 5000 に大幅に入力が削減されて再開
[8/9]  input: 13,000
  ...
```

#### Compaction 発動時のレスポンス構造

`stop_reason` が通常の `"end_turn"` ではなく **`"compaction"`** になり、`content[0]` に要約結果が格納されます。input_tokensとoutput_tokensはそれぞれCompaction実行時に入力したトークン数と、要約を出力した際のトークン数を指します:

```json
{
  "content": [
    {
      "type": "compaction",
      "content": "これまでの会話の要約: ユーザーはWebアプリケーション開発の..."
    }
  ],
  "stop_reason": "compaction",
  "usage": {
    "input_tokens": 53000, 
    "output_tokens": 2000
  }
}
```

#### クライアント側のハンドリング

Compaction レスポンスを受け取ったら、クライアントは **compaction ブロック + 直近のメッセージ** で messages を再構築して API を再呼び出しします。このパターンは [公式ドキュメント](https://platform.claude.com/docs/en/build-with-claude/compaction) でも紹介されている方法です。

このサンプルコードでは `compaction_messages[-2:]` で直近 **2 メッセージ** を保持しています。例えば 7 回目の質問で発動した場合:

```python
# compaction_messages = [user1, asst1, ..., user6, asst6, user7]
# compaction_messages[-2:] → [asst6, user7]

messages_after = [
    {"role": "assistant", "content": [compaction_block]},    # 以前の会話の要約
    {"role": "assistant", "content": [...]},                 # 6回目のClaudeの回答 (asst6)
    {"role": "user",      "content": "docker-composeで..."}, # 7回目のユーザーからの質問 (user7)
]
```

5回目以前のやりとりは compaction ブロックの要約にのみ含まれ、原文は破棄されます。API は compaction ブロックより前のメッセージを自動的に無視するため、要約 + 直近の原文で会話の連続性を保ちつつ、入力トークンを大幅に削減できます。

---
## まとめ

このノートブックで扱った機能を振り返ります。

- Adaptive Thinking + Effort
    - リクエストの複雑さに応じて Claude が自律的に思考の深さを調整する機能。effortパラメータで思考量のガイダンスを指定可能                                                                                                
- 1M コンテキストウィンドウ (Beta)
    - デフォルト 200K トークンのコンテキストウィンドウを 1Mトークンに拡張し、大量のドキュメントやコードベースを一度に処理可能にする                                     
- 最大 128K の出力トークン
    - 出力トークンの上限を最大 128K トークンまで指定可能。長文のコード生成や翻訳、レポート作成に有用                  
- 構造化出力 - JSON Outputs
    - LLM のレスポンスを指定した JSON Schema に厳密に準拠させ、型安全でパースエラーのない出力を保証する機能
- Compaction (Beta)
    - 長い会話でコンテキストウィンドウのトークン数がしきい値を超えた際に、会話を自動要約して圧縮し、会話を継続可能にする機能

---

### Appendix - Converse API で実装するパターンの設定方法

InvokeModel API を利用する場合、Amazon Bedrock で利用可能な全ての Claude の機能が利用できる。Converse APIでも主要な機能は対応しているが、2026年2月時点で  Converse API は以下の機能が未対応となっている。

- Compaction: ValidationException: The compact beta feature is not currently supported on the Converse and ConverseStream APIs となり失敗
- Tool Search Tool: The tool-search-tool beta feature is not currently supported on the Converse and ConverseStream APIs となり失敗
- Document Citations: PDFのみ対応、txtは現状失敗する

```python
bedrock.converse(
    modelId=MODEL_ID,
    messages=[...], # ここに各種パラメータを指定
    inferenceConfig={"maxTokens": ...},
    additionalModelRequestFields={...},  # ここに各種パラメータを指定
)
```

#### Adaptive Thinking + Effort
```python
additionalModelRequestFields={
    "thinking": {"type": "adaptive"},
    "output_config": {"effort": "high"},
}
```

#### Structured Outputs
```python
additionalModelRequestFields={
    "output_config": {"format": {"type": "json_schema", "schema": {...}}},
}
```

#### 1M コンテキスト
```python
additionalModelRequestFields={
    "anthropic_beta": ["context-1m-2025-08-07"],
}
```

#### Context Editing
```python
additionalModelRequestFields={
    "anthropic_beta": ["context-management-2025-06-27"],
    "context_management": {
        "edits": [{
            "type": "clear_tool_uses_20250919",
            "trigger": {"type": "input_tokens", "value": 5000},
            "keep": {"type": "tool_uses", "value": 2},  # invoke_model では整数、Converseではdict
        }]
    },
}
```

#### Computer Use / Memory Tool / Bash Tool (schema-less)
```python
additionalModelRequestFields={
    "anthropic_beta": ["computer-use-2025-11-24"],  # Computer Useの場合
    "tools": [{"type": "computer_20251124", "name": "computer", ...}],
}
```

#### Fine-grained Tool Streaming — toolConfig は使わず additionalModelRequestFields のみ
```python
additionalModelRequestFields={
    "tools": [{"name": "...", "input_schema": {...}, "eager_input_streaming": True}],
    "tool_choice": {"type": "any"},
}
```

#### Prompt Caching — cachePoint (invoke_model の cache_control とは別形式)
```python
messages=[
    {"text": "..."},
    {"cachePoint": {"type": "default"}},          # 5分TTL
    # {"cachePoint": {"type": "default", "ttl": "1h"}},  # 1時間TTL
]
```

### Citations (searchResult) 
```python
messages=[{"role": "user", "content": [
    {"searchResult": {"source": "URL", "title": "...", "content": [...], "citations": {"enabled": True}}},
    {"text": "質問"},
]}]
```

#### Citations (PDF document)
```python
{"document": {"name": "doc", "format": "pdf", "source": {"bytes": ...}, "citations": {"enabled": True}}}
```